In [1]:
import numpy as np
from abc import ABC, abstractmethod

# SOLID

Принципи за писане на код; дават:
- лесна бъдеща разширимост
- простота
- повече абстракция

## 1 Single Responsibility

* деф: всеки клас, модул или фунцкия да
  прави само едно нещо
* единствена отговорност
* публичен интерфейс = това (методите)
които се виждат отвън

In [4]:
# Bad
class Student:
    def __init__(self, name):
        self.name = name

    def get_name(self):
        return self.name

    def register(self, student):
        pass

In [5]:
# Good
class Student:
    def __init__(self, name):
        self.name = name

    def get_name(self):
        return self.name
    

class StudentRegistry:
    def __init__(self):
        self.students = []
        
    def register(self, student):
        self.students.append(student)

In [9]:
# Bad
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.page = 0

    def turn_page(self, page):
        self.page = page


class Library:
    def __init__(self, books):
        self.books = books

    def add_book(self, book):
        self.books.append(book)

In [35]:
# Good
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.page = 0


class BookInstance:
    def __init__(self, book):
        self.book = book
        self.page = 0
    
    def turn_page(self, page):
        self.page = page


class Library:
    def __init__(self, books):
        self.books = books

    def add_book(self, book):
        self.books.append(book)

## 2 Open/Closed

* класовете, модулите и функциите
  трябва да са отворени за разширение,
  но затворени за модификация
* може да се постигне чрез: абстракция,
миксини, monkey patching
* Monkey patching = пренаписване на 
метод извън класа с ламбда фунцкия
* monkey patching се ползва при тестване

### Student taxes

Initial code is below. Later `get_discont` is modified to add a new discount case.

#### Bad

In [12]:
# Bad - OCP violation
class StudentTaxes:
    def __init__(self, name, semester_tax, average_grade):
        self.name = name
        self.semester_tax = semester_tax
        self.average_grade = average_grade

    def get_discount(self):
        if self.average_grade > 5:
            return self.semester_tax * 0.4
        

class StudentTaxes:
    def __init__(self, name, semester_tax, average_grade):
        self.name = name
        self.semester_tax = semester_tax
        self.average_grade = average_grade

    def get_discount(self):
        if self.average_grade > 5:
            return self.semester_tax * 0.4
        elif self.average_grade > 4:
            return self.semester_tax * 0.2

#### Good

In [14]:
# Good
class StudentTaxes:
    def __init__(self, name, semester_tax, average_grade):
        self.name = name
        self.semester_tax = semester_tax
        self.average_grade = average_grade

    def get_discount(self):
        return 0 
    
class ExcellentStudentTaxes(StudentTaxes):
    def get_discont(self):
        if self.average_grade >= 5:
            return self.semester_tax * 0.4

#### Good - reformat to satisfy SRP

In [30]:
# Even better
class Student:
    def __init__(self, name, grades):
        self.name = name
        self.grades = grades
    
    @property
    def average_grade(self):
        return np.mean(self.grades)


class StudentTaxes:
    def __init__(self, student, semester_tax):
        self.student = student
        self.semester_tax = semester_tax

    def get_discount(self):
        return 0 
    

class ExcellentStudentTaxes(StudentTaxes):
    def get_discount(self):
        if self.student.average_grade >= 5:
            return self.semester_tax * 0.4
        return super().get_discount()
    
    
class GoodStudentTaxes(StudentTaxes):
    def get_discount(self):
        if self.student.average_grade >= 4:
            return self.semester_tax * 0.2
        return super().get_discount()

In [32]:
gosho = Student("Gosho", [4, 5])
emil = Student("Emil", [5, 6])

gosho_discount = GoodStudentTaxes(gosho, 100)
emil_discount = ExcellentStudentTaxes(emil, 100)

print(gosho_discount.get_discount())
print(emil_discount.get_discount())

20.0
40.0


#### Good - use single class

In [33]:
# Even better
class Student:
    def __init__(self, name, grades):
        self.name = name
        self.grades = grades
    
    @property
    def average_grade(self):
        return np.mean(self.grades)


class StudentTaxes:
    def __init__(self, student, semester_tax):
        self.student = student
        self.semester_tax = semester_tax

    def get_discount(self):
        return 0 
    

class ExcellentStudentTaxes(StudentTaxes):
    def get_discount(self):
        if self.student.average_grade >= 5:
            return self.semester_tax * 0.4
        return super().get_discount()
    
    
class GoodStudentTaxes(ExcellentStudentTaxes):
    def get_discount(self):
        current_discount = 0
        if self.student.average_grade >= 4:
            current_discount = self.semester_tax * 0.2
        return max(current_discount, super().get_discount())

In [34]:
gosho = Student("Gosho", [4, 5])
emil = Student("Emil", [5, 6])

gosho_discount = GoodStudentTaxes(gosho, 100)
emil_discount = GoodStudentTaxes(emil, 100)

print(gosho_discount.get_discount())
print(emil_discount.get_discount())

20.0
40.0


#### Good - use abstract class

In [33]:
# Even better
class Student:
    def __init__(self, name, grades):
        self.name = name
        self.grades = grades
    
    @property
    def average_grade(self):
        return np.mean(self.grades)


class StudentTaxes(ABC):
    def __init__(self, student, semester_tax):
        self.student = student
        self.semester_tax = semester_tax
    
    @abstractmethod
    def get_discount(self):
        pass
    

class ExcellentStudentTaxes(StudentTaxes):
    def get_discount(self):
        if self.student.average_grade >= 5:
            return self.semester_tax * 0.4
        return super().get_discount()
    
    
class GoodStudentTaxes(ExcellentStudentTaxes):
    def get_discount(self):
        current_discount = 0
        if self.student.average_grade >= 4:
            current_discount = self.semester_tax * 0.2
        return max(current_discount, super().get_discount())

### Animals

#### Bad

In [ ]:
class Animal:
    def __init__(self, species):
        self.species = species

    def get_species(self):
        return self.species


def animal_sound(animals: list):
    for animal in animals:
        if animal.species == 'cat':
            print('meow')
        elif animal.species == 'dog':
            print('woof-woof')


animals = [Animal('cat'), Animal('dog')]
animal_sound(animals)

## добавете ново животно и рефакторирайте кода да работи без да се налага да се правят промени по него
## при добавяне на нови животни
# animals = [Animal('cat'), Animal('dog'), Animal('chicken')]

#### Better

In [42]:
class Animal(ABC):
    @abstractmethod
    def make_sound():
        pass


class Cat(Animal):
    sound = "mew"
    def make_sound(self):
        return self.sound

    
class Dog(Animal):
    sound = "ruf"
    def make_sound(self):
        return self.sound


def animal_sound(animals: list):
    for animal in animals:
        print(animal.make_sound())


animals = [Cat(), Dog()]
animal_sound(animals)

mew
ruf


## 3 Liskov Substitution

* всеки наследник да може да се ползва като родител
* т.е. наследниците да не премахват или променят методите / атрибутите на родителя
* всяка котка може да прави каквото прави животното
* ако наследяваме клас и не използваме всичко от класа - нарушение на Лисков. Може да се изкара нов родител.

### Animals

#### Bad

In [44]:
class Animal(ABC):
    @abstractmethod
    def make_sound():
        pass


class Cat(Animal):
    sound = "mew"
    def make_sound(self):
        return self.sound

    
class Dog(Animal):
    sound = "ruf"
    def make_sound(self):
        return self.sound

    
class Mole(Animal):
    def make_sound(self):
        raise TypeError("Moles do not make sounds")
    

def animal_sound(animals: list):
    for animal in animals:
        print(animal.make_sound())


animals = [Cat(), Dog(), Mole()]
# animal_sound(animals)  # TypeError: Moles do not make sounds

#### Good

In [48]:
class Animal:
    pass


class SoundMakingAnimal(Animal, ABC):
    @abstractmethod
    def make_sound():
        pass


class Cat(SoundMakingAnimal):
    sound = "mew"
    def make_sound(self):
        return self.sound

    
class Dog(SoundMakingAnimal):
    sound = "ruf"
    def make_sound(self):
        return self.sound

    
class Mole(Animal):
    pass
    

def animal_sound(animals: list):
    for animal in animals:
        print(animal.make_sound())


animals = [Cat(), Dog()]
animal_sound(animals)  # TypeError: Moles do not make sounds

mew
ruf


## 4 Interface Segregation

* един клас не трябва да имплементира методи, които не използва 
* един клас не трябва да наследява функционалност, която не му трябва

### Entertainment devices

#### Bad

In [ ]:
class EntertainmentDevice:
    def connect_to_device_via_hdmi_cable(self, device): pass
    def connect_to_device_via_rca_cable(self, device): pass
    def connect_to_device_via_ethernet_cable(self, device): pass
    def connect_device_to_power_outlet(self, device): pass


class Television(EntertainmentDevice):
    def connect_to_dvd(self, dvd_player):
        self.connect_to_device_via_rca_cable(dvd_player)

    def connect_to_game_console(self, game_console):
        self.connect_to_device_via_hdmi_cable(game_console)

    def plug_in_power(self):
        self.connect_device_to_power_outlet(self)


class dvd_player(EntertainmentDevice):
    def connect_to_tv(self, television):
        self.connect_to_device_via_hdmi_cable(television)

    def plug_in_power(self):
        self.connect_device_to_power_outlet(self)



class GameConsole(EntertainmentDevice):
    def connect_to_tv(self, television):
        self.connect_to_device_via_hdmi_cable(television)

    def connect_to_router(self, router):
        self.connect_to_device_via_ethernet_cable(router)

    def plug_in_power(self):
        self.connect_device_to_power_outlet(self)


class Router(EntertainmentDevice):
    def connect_to_tv(self, television):
        self.connect_to_device_via_ethernet_cable(television)

    def connect_to_game_console(self, game_console):
        self.connect_to_device_via_ethernet_cable(game_console)

    def plug_in_power(self):
        self.connect_device_to_power_outlet(self)

#### Good

In [50]:
class HdmiConnectable:
    def connect_via_hdmi(self, device):
        pass

    
class RcaConnectable:
    def connect_via_rca(self, device):
        pass

    
class EthernetConnectable:
    def connect_via_ethernet(self, device):
        pass

    
class EntertainmentDevice:
    def plug_in_power(self, device):
        pass


class Television(EntertainmentDevice, HdmiConnectable, RcaConnectable):
    def connect_to_dvd(self, dvd_player):
        self.connect_via_rca(dvd_player)

    def connect_to_game_console(self, game_console):
        self.connect_to_hdmi(game_console)


class GameConsole(EntertainmentDevice, HdmiConnectable, EthernetConnectable):
    def connect_to_tv(self, television):
        self.connect_via_hdmi(television)

    def connect_to_router(self, router):
        self.connect_via_ethernet(router)


class Router(EntertainmentDevice, EthernetConnectable):
    def connect_to_tv(self, television):
        self.connect_via_ethernet(television)

    def connect_to_game_console(self, game_console):
        self.connect_via_ethernet(game_console)

## 5 Dependency Inversion

* Не разчитаме на конкретни неща, а на абстракция.
* Логиката на метода да е абстрактна. Т.е. алгоритъмът съдържа стъпките ибори с параметри. Конкретиката кой форматр ползва е изкарана отвън.

### Workers

In [57]:
class Worker:
    def work(self):
        print("I'm working!!!")


class SuperWorker():
    pass


class Manager:
    def __init__(self):
        self.workers = []
        
    def set_worker(self, worker):
        self.workers.append(worker)
    
    def manage(self):
        for worker in self.workers: 
            worker.work()



worker = Worker()
manager = Manager()
manager.set_worker(worker)
manager.manage()

super_worker = SuperWorker()
try:
    manager.set_worker(super_worker)
except AssertionError:
    print("manager fails to support super_worker....")

I'm working!!!
